In [88]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re



steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, \
                'Content_Focus': content_focus_df, 'Component': component_df}

In [119]:
#filename = "math sample for API testing.csv"
filename = "Math Tagging Master File 3.4.2017.csv"
#filename = "Math Test.csv"
#filename = "0 step test.csv"

In [120]:
parsed_data, error_data, component_holding, content_focus_component_df = process_file(filename, tagging_dict)
#pprint.pprint(parsed_data)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000


In [121]:
print len(error_data)

2334


In [122]:
print len(parsed_data)

33558


In [123]:
print len(component_holding)

533


In [125]:
#pprint.pprint(error_data)

In [103]:
def process_file(filename, tagging_df):
    parsed_data = []
    error_data = []
    component_holding = []
    initial_components = True
    content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        counter = 0
        for line in reader:
            counter += 1
            if counter % 500 == 0:
                print counter
            error_test = False
            component_list = line['Component'].split("; ")
            content_focus_list = line['Content Focus'].split("; ")
            #first pass, only process those items that have one component or only one content focus, build data frame of 
            #allowable strand/substrand/content focus/component combinations
            if len(component_list) == 1 or len(content_focus_list) == 1:
                row, error_test, component_holding, content_focus_component_df = parse_line( \
                    line, tagging_df, content_focus_list, component_list, error_test, component_holding, \
                    content_focus_component_df, initial_components)
                if error_test == True:
                    error_data.append(row)                
                else: parsed_data.append(row)
                if component_holding == True:
                    component_holding.append(line)
            else: component_holding.append(line)
    initial_components = False
    #process items sent to component_holding
    return parsed_data, error_data, component_holding, content_focus_component_df

In [58]:
def parse_line(line, tagging_df, content_focus_list, component_list, error_test, component_holding, \
               content_focus_component_df, initial_components):
    row = {}
    strand_dict = {}
    strand_list = []
    differentiator_list = []
    content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    InternalID = int(line['Internal ID'])
    #print InternalID
    Strand = line['Strand'].split("; ")
    for strand in Strand:
        strand_sub_dict = {}
        content_focus_df = tagging_df['Content_Focus']
        GUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
        strand_sub_dict = {'Strand': strand, 'GUID' : GUID, 'Substrand': []}
        strand_list.append(strand_sub_dict)
    if error_test == False:
        substrand_list = line['Sub-Strand'].split("; ")
        strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)
        if error_test == False:
            strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, \
                                                            error_test)
            if error_test == False and initial_components == True:
                component_hold = False
                strand_list, error_test, component_hold, content_focus_component_df = processInitialComponent(\
                    strand_list, substrand_list, content_focus_list, component_list, tagging_df['Component'], error_test, \
                    component_hold, content_focus_component_df)
            #add 2nd pass component processing
            if error_test == False:
                differentiator_keys = ['Shape', 'Units', 'Steps']            
                for differentiator in differentiator_keys:
                    differentiator_sub_list = []
                    differentiator_df = tagging_df[differentiator]
                    differentiator_values = line[differentiator].split("; ")
                    for value in differentiator_values:
                        differentiator_sub_dict = {}
                        GUID, error_test = get_GUID_lite(value, differentiator_df, error_test)
                        differentiator_sub_dict = {differentiator: value, 'GUID': GUID}
                        differentiator_sub_list.append(differentiator_sub_dict)
                        if error_test == True:
                            break
                    differentiator_list.append(differentiator_sub_list)
    row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": differentiator_list}
    return row, error_test, component_holding, content_focus_component_df

In [70]:
def processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, component_hold, \
                            content_focus_component_df):
    GUID = None
    for component in component_list:
        if component == 'N/A':
            pass
        elif len(df[(df['Title']==lowercase(component))]) == 0:  
            error_test = True
            error_message = 'Error: Component ' + component + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, \
                                                                                    'GUID': error_message})
            break
    if len(content_focus_list) == 1:
        for component in component_list:
            strand_list, error_test, component_hold, guid, content_focus_component_df = getComponentGUID( \
                strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus_list[0], component, df, error_test, \
                component_hold, content_focus_component_df) 
            #add to content_focus_component_df
    elif len(substrand_list) == len(component_list):
        strand_list, error_test, component_hold, content_focus_component_df = matchInitialComponent(strand_list, substrand_list,\
                            content_focus_list, component_list, df, error_test, len(component_list), content_focus_component_df)            
    else: 
        component_hold = True
    return strand_list, error_test, component_hold, content_focus_component_df

def getComponentGUID(strand, strand_list, substrand, content_focus, component, df, error_test, component_hold, \
                     content_focus_component_df): 
    ComponentGUID = ""
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate( \
            strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    content_focus_index = next(index for (index, d) in enumerate( \
            strand_list[strand_index]['Substrand'][substrand_index]['Content Focus']) if d["Content Focus"] == content_focus)
    if lowercase(component) == "n/a":
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'N/A'})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, 'N/A')
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
              (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component)),\
                              'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & 
                (df['Title']==lowercase(component))]) > 1:
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Holding'})
        component_hold = True
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Error: No Strand/Substrand/Component match found in master list'})
        error_test = True
    return strand_list, error_test, component_hold, ComponentGUID, content_focus_component_df

def matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, matchlimit, \
                          content_focus_component_df):
    component_hold = False
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=component_list)
    strand_substrand_names[0].split('; ')[0]

    for strand_substrand in strand_substrand_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for component in component_list:
            initial_match_length = len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                                          (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& \
                                          (df['Title']==lowercase(component))])
            if  initial_match_length == 1:
                match_status.ix[strand_substrand, component] = True
            elif initial_match_length > 1:
                    if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                              (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1])) & \
                              (df['Example Content Focus']==lowercase(content_focus_list[0])) & \
                              (df['Title']==lowercase(component))]) == 1:
                        match_status.ix[strand_substrand, component] = True
                
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test, component_hold, GUID, content_focus_component_df = getComponentGUID(
                pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1], content_focus_list[0], pair[1], component_df, \
                error_test, component_hold, content_focus_component_df)
    else: 
        #strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': 'Error: Component not found'})
        component_hold = True
    return strand_list, error_test, component_hold, content_focus_component_df


def checkContentFocusComponent(strand, substrand, content_focus, component, content_focus_component_df, GUID):
    if len(content_focus_component_df[(content_focus_component_df['ComponentGUID']==GUID)]) == 0:
        #add group to content_focus_component_df
        df_add = {'Strand': lowercase(strand), 'Sub-Strand': lowercase(substrand), \
                  'Content Focus': lowercase(content_focus), "Component": lowercase(component), \
                  "ComponentGUID": GUID}
        content_focus_component_df = content_focus_component_df.append(df_add, ignore_index = True)
    return content_focus_component_df

In [31]:
def processSubstrands(strand_list, substrand_list, df, error_test):
    GUID = None
    for substrand in substrand_list:
        if len(df[(df['Title']==lowercase(substrand))]) == 0:  
            error_test = True
            error_message = 'Error: Substrand ' + substrand + ' not found in master list'
            strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': error_message})
            break
    if len(substrand_list) < len(strand_list):
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': 'Error: more strands than substrands'})
    elif len(strand_list) == 1: 
        for substrand in substrand_list:
            strand_list, error_test = getSubstrandGUID(strand_list[0]['Strand'], strand_list, substrand, df, error_test)    
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': \
                                            'Error: Multiple substrands, cannot parse yet'})
    return strand_list, error_test

#Build a data frame to track match status to determine whether strand-substrand pairings can be deduced    
def matchStrandsSubstrands(strand_list, substrand_list, df, error_test, matchlimit):
    strand_names = []
    for strand in strand_list:
        strand_names.append(strand['Strand'])

    match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)

    for strand in strand_names:
        for substrand in substrand_list:
            if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
                match_status.ix[strand,substrand] = True
    
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        for pair in true_pairs:
            strand_list, error_test = getSubstrandGUID(pair[0], strand_list, pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], \
                                            'GUID': 'Error: Ambiguous strand/substrand relationships'})
        error_test = True
    return strand_list, error_test

def getSubstrandGUID(strand, strand_list, substrand, df, error_test): 
    if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
        SubstrandGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append( \
            {'Substrand': substrand, 'GUID': SubstrandGUID, 'Content Focus': []})
    else: 
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append( \
            {'Substrand': substrand, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

In [51]:
def processContentFocus(strand_list, substrand_list, content_focus_list, df, error_test):
    GUID = None
    for content_focus in content_focus_list:
        if len(df[(df['Title']==lowercase(content_focus))]) == 0:  
            error_test = True
            error_message = 'Error: Content Focus ' + content_focus + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': error_message})
            break
    if len(content_focus_list) < len(substrand_list):
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                                'Error: more Substrands than Content Focus terms'})
    elif len(substrand_list) == 1: 
        for content_focus in content_focus_list:
            strand_list, error_test = getContentFocusGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], \
                                                          content_focus, df, error_test)    
    else:
        strand_list, error_test = matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, \
                                                              len(content_focus_list))
    return strand_list, error_test

def matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, matchlimit):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=content_focus_list)
    for strand_substrand in strand_substrand_names:
        for content_focus in content_focus_list:
            #print strand_substrand
            #print content_focus
            if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                      (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& \
                      (df['Title']==lowercase(content_focus))]) == 1:
                match_status.ix[strand_substrand, content_focus] = True

    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test = getContentFocusGUID(pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1],  \
                                                          pair[1], df, error_test)
    elif int(match_status.sum().sum()) < matchlimit:
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                                'Error: One or more content focus terms not found'})
        error_test = True
    else: 
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': \
                                                            'Error: Ambiguous strand/substrand/content focus relationships'})
        error_test = True
    return strand_list, error_test

def getContentFocusGUID(strand, strand_list, substrand, content_focus, df, error_test): 
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate( \
                                                     strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
              (df['Title']==lowercase(content_focus))]) == 1:
        ContentFocusGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                                 (df['Title']==lowercase(content_focus)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append( \
                            {'Content Focus': content_focus, 'GUID': ContentFocusGUID, 'Component': []})
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append( \
                            {'Content Focus': content_focus, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

In [8]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [9]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
        df_lowercase(data, 'Example Content Focus')
    return data

In [10]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [112]:
def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        elif tag == '0':
            GUID = "Field not tagged"
        else: 
            GUID = "Error: no matches found"
            error_test = True
    else: 
        GUID = "Error: resolve multiple values"
        error_test = True
    return GUID, error_test

In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: 
                        {Number: [N1, N2, ...]}
                            {Digits: [D1, D2, ...]}
                        {Shape: [Shape1, ...]}
                        {Units: [Units1, ...]}
                        {Steps: Steps}
                        {Context: Context}
                        {SIG Stem: [Sig Stem1, ...]}
                        {SIG Options: [SIG Options1, ...]}
                        {Additional Modifiers: [AM1, ...]}
}

'''

Scratch area

In [ ]:
df = component_df
strand = 'Geometry and Measurement'
substrand = 'Coordinate Plane'
component = 'First Quadrant'
content_focus = 'Graph Points'
content_focus_component_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
ComponentGUID = 'test'
content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
print content_focus_component_df
#print df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component))]

In [ ]:
def matchHeldComponent(strand_list, substrand_list, content_focus_list, component_list, error_test, content_focus_component_df):
    component_hold = False
    strand_substrand_cf_names =[]
    for strand in strand_list:
        for substrand in strand['Substrand']:
            for content_focus in content_focus_list:
                strand_substrand_cf_names.append(strand['Strand'] + '; ' + substrand['Substrand'] + '; ' +\
                                                 content_focus)

    match_status = pd.DataFrame(False,index=strand_substrand_cf_names,columns=component_list)
#    strand_substrand_names[0].split('; ')[0]

    for strand_substrand_cf in strand_substrand_cf_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for component in component_list:
           
            elif initial_match_length > 1:
                    if len(content_focus_component_df[\
                            (content_focus_component_df['Strand']==lowercase(strand_substrand.split('; ')[0])) & \
                            (content_focus_component_df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1])) & \
                              (content_focus_component_df['Content Focus']==lowercase(content_focus_list[0])) & \
                              (content_focus_component_df['Component']==lowercase(component))]) == 1:
                        match_status.ix[strand_substrand_cf, component] = True
                
    if int(match_status.sum().sum()) == len(component_list:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
#            strand_list, error_test, component_hold, GUID, content_focus_component_df = getComponentGUID(
#                pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1], content_focus_list[0], pair[1], component_df, \
#                error_test, component_hold, content_focus_component_df)
    else: 
        #strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': 'Error: Component not found'})
        component_hold = True
    return strand_list, error_test, component_hold, content_focus_component_df

def getHeldComponentGUID(strand, substrand, content_focus, component, error_test, content_focus_component_df): 
    ComponentGUID = ""

    ******Stopped here***********                                        
    if lowercase(component) == "n/a":
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'N/A'})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, 'N/A')
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
              (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & \
                              (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component)),\
                              'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': ComponentGUID})
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & 
                (df['Title']==lowercase(component))]) > 1:
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Holding'})
        component_hold = True
        content_focus_component_df = checkContentFocusComponent(strand, substrand, content_focus, component, \
                                                             content_focus_component_df, ComponentGUID)
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append( \
            {'Component': component, 'GUID': 'Error: No Strand/Substrand/Component match found in master list'})
        error_test = True
    return strand_list, error_test, component_hold, ComponentGUID, content_focus_component_df

In [ ]:
Note to self: Need to take care of case where fewer Components than Content Focus (Likely N/A)